Conteo de palabras en Apache Hive en modo standalone
===

* *30 min* | Última modificación: Noviembre 14, 2019

En este tutorial se aborda el proceso de construcción de aplicaciones en Apache Hive.

## Ejecución de Hive en un contenedor de Docker

* Usando el directorio de trabajo de la máquina local:

```
docker run --rm -it -v "$PWD":/datalake  --name hive -p 8888:8888 jdvelasq/hive:2.3.6-standalone
```

* Usando un volumen de Docker (llamado `datalake`):

```
docker run --rm -it -v datalake:/datalake --name hive -p 8888:8888 jdvelasq/hive:2.3.6-standalone
```


* Consola conectada a un contendor que ya está corriendo:

```
docker exec -it hive bash
```


## Carga de la librería de Jupyter

Esta librería permite enviar comandos a Hive de forma interactiva.

In [1]:
%load_ext bigdata

In [2]:
%hive_start

In [3]:
%timeout 300

## Codificación y pruebas

En esta primera parte se aborda el proceso de desarrollo y depuración de una aplicación de Hive en la máquina local.

### Generación de datos de prueba

Los datos que usa la aplicación se encuentran localizados como una carpeta en el directorio actual de trabajo. Estos datos serán consumidos por Hive directamente.

In [4]:
## 
## Se crea el directorio wordcount en la carpeta actual de trabajo
## y se escriben tres archivos en ella.
##
!mkdir -p wordcount/

A continuación se generan tres archivos de prueba que se almacenan en la carpeta `wordcount/`.

In [5]:
%%writefile wordcount/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Overwriting wordcount/text0.txt


In [6]:
%%writefile wordcount/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Overwriting wordcount/text1.txt


In [7]:
%%writefile wordcount/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Overwriting wordcount/text2.txt


### Creación de tablas

En esta aplicación se usarán dos tablas:

* `docs`: para cargar el contenido de los archivos de texto, donde cada línea equivale a un registro.

* `word_counts`: En donde aparece cada palabra y su respectivo conteo.

A continuación se elimnan dichas tablas si existen en el sistema, y luego se crea la tabla `docs` con un solo campo del tipo `STRING`.

In [8]:
%%hive
DROP TABLE IF EXISTS docs;
DROP TABLE IF EXISTS word_counts;
CREATE TABLE docs (line STRING);

DROP TABLE IF EXISTS docs;
OK
Time taken: 6.618 seconds
DROP TABLE IF EXISTS word_counts;
OK
Time taken: 0.008 seconds
CREATE TABLE docs (line STRING);
OK
Time taken: 0.96 seconds


### Carga de datos

En el siguiente código, se hace la carga directa de todos los archivos que se encuentran en el directorio `wordcount` en la tabla `docs`. Luego, se imprimen los primeros cinco registros de la tabla para verificar que la lectura fue correcta.

In [9]:
%%hive
LOAD DATA LOCAL INPATH "wordcount/" OVERWRITE INTO TABLE docs;
SELECT * FROM docs LIMIT 5;

LOAD DATA LOCAL INPATH "wordcount/" OVERWRITE INTO TABLE docs;
Loading data to table default.docs
OK
Time taken: 1.173 seconds
SELECT * FROM docs LIMIT 5;
OK
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Time taken: 2.242 seconds, Fetched: 5 row(s)


### Transformación de datos

Una vez cargados los archivos, se procede a partir las líneas por palabras, usando la función `split(line, '\\s')`;  la expresión `\\s` indica que se realice la partición por los espacios en blanco; de esta forma, `split()` genera una lista de palabras. La función `explode(.)` de Hive en conjunto con `SELECT`, genera un nuevo registro por cada palabra en `line`. 

In [10]:
%%hive
SELECT explode(split(line, '\\s')) AS word FROM docs LIMIT 5;

SELECT explode(split(line, '\\s')) AS word FROM docs LIMIT 5;
OK
Analytics
is
the
discovery,
interpretation,
Time taken: 0.688 seconds, Fetched: 5 row(s)


#### Conteo de palabras

Para realizar el conteo, la expresión `SELECT word, count(1) AS count ... GROUP BY word` genera una tabla con dos columnas, donde la primera columna (`word`) correspodne a cada palabra en el texto, y la segunda columna representa la cantidad de veces que aparece en los registros generados por la expresión `SELECT explode(split(line, '\\s')) AS word FROM docs`. 

In [11]:
%%hive
CREATE TABLE word_counts 
AS
    SELECT word, count(1) AS count 
    FROM
        (SELECT explode(split(line, '\\s')) AS word FROM docs) w
GROUP BY 
    word
ORDER BY 
    word;

CREATE TABLE word_counts 
AS
    SELECT word, count(1) AS count 
    FROM
        (SELECT explode(split(line, '\\s')) AS word FROM docs) w
GROUP BY 
    word
ORDER BY 
    word;
Query ID = root_20191115154030_a472520b-a145-439e-902b-a7adae054ec1
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2019-11-15 15:40:32,861 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local2011660276_0001
Launching Job 2 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum 

### Resultado

Para visualizar los resultados obtenidos, se realiza un `SELECT` sobre la tabla `word_counts`.

In [12]:
%%hive
SELECT * FROM word_counts LIMIT 10;

SELECT * FROM word_counts LIMIT 10;
OK
	20
(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Time taken: 0.137 seconds, Fetched: 10 row(s)


### Cierre de Hive

Finalmente, y una vez se ha terminado de depurar el código, se cierra el interprete de Hive que se abrió en el background.

In [13]:
%hive_quit

## Versión en productivo

En la segunda parte, se procede a llevar el aplicativo a productivo con los siguientes cambios:

* Los datos son leidos del sistema HDFS de Hadoop.

* Los resultdos son guardados en una carpeta del sistema Hadoop.

* El script se almacena en un archivo en el disco duro, para su uso posterior.

### Copia de los datos al sistema HDFS

In [14]:
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 5 items
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/hadoop-root
drwxrwxrwx   - root root       4096 2019-11-15 15:40 /tmp/hive
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/hsperfdata_root
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/root
-rw-r--r--   1 root root          0 2019-11-15 15:35 /tmp/stderr


In [15]:
##
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

In [16]:
##
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 6 items
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/hadoop-root
drwxrwxrwx   - root root       4096 2019-11-15 15:40 /tmp/hive
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/hsperfdata_root
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/root
-rw-r--r--   1 root root          0 2019-11-15 15:35 /tmp/stderr
drwxr-xr-x   - root root       4096 2019-11-15 15:40 /tmp/wordcount


In [17]:
##
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal wordcount/*  /tmp/wordcount/

In [18]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 3 items
-rw-r--r--   1 root root       1093 2019-11-15 15:40 /tmp/wordcount/text0.txt
-rw-r--r--   1 root root        352 2019-11-15 15:40 /tmp/wordcount/text1.txt
-rw-r--r--   1 root root        440 2019-11-15 15:40 /tmp/wordcount/text2.txt


### Generación del script y ajuste del código

Se realizan dos cambios. En primer lugar, se sustituye la línea 

    LOAD DATA LOCAL INPATH "wordcount/" OVERWRITE INTO TABLE docs;
    
por:

    LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE docs;

para que Hive lea los datos del directorio `/tmp/wordcount/` en el HDFS. En segundo lugar, se agrega

    INSERT OVERWRITE DIRECTORY '/tmp/output' 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
    SELECT * FROM word_counts;
    
para que los resultados sean almacenados en la carpeta `/tmp/output` como un archivo en formato CSV. El programa es guadado como `wordcount.hql` en el computador local. 

In [19]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS docs;
DROP TABLE IF EXISTS word_counts;

CREATE TABLE docs (line STRING);

LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE docs;

CREATE TABLE word_counts 
AS
    SELECT word, count(1) AS count 
    FROM
        (SELECT explode(split(line, '\\s')) AS word FROM docs) w
GROUP BY 
    word
ORDER BY 
    word;
    
INSERT OVERWRITE DIRECTORY '/tmp/output' 
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
SELECT * FROM word_counts;


Writing wordcount.hql


### Ejecución

Una vez se ha almacenado el programa, se puede ejecutar con:

In [20]:
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


donde `-S` indica que Hive se ejecute en modo silencioso; `-e` que se ejecute la expresión `source wordcount.hql`. Lo anterior es equivalente a abrir Hive y luego ejecutar:

     hive> source 'wordcount.hql'
     
     

### Visualización de resultados

Los resultados quedan almacenados en la carpeta `/tmp/output` del sistema HDFS

In [21]:
## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/output

Found 1 items
-rwxrwxrwt   1 root root       1653 2019-11-15 15:41 /tmp/output/000000_0


In [22]:
## se visualiza la cabecera del archivo.
!hdfs dfs -cat /tmp/output/000000_0 | head

,20
(DA),1
(see,1
Analytics,2
Analytics,,1
Big,1
Data,3
Especially,1
Organizations,1
Since,1


### Copia de los resultados a la máquina local

In [23]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [24]:
!cat output/000000_0

,20
(DA),1
(see,1
Analytics,2
Analytics,,1
Big,1
Data,3
Especially,1
Organizations,1
Since,1
Specifically,,1
The,2
a,1
about,1
aid,1
algorithms,1
analysis,,1
analysis.,1
analytics,8
analytics,,8
analytics.,1
analyze,1
and,15
application,1
apply,1
are,1
areas,2
assortment,1
be,1
big,1
business,4
by,2
call,1
can,2
certain,1
changes.,1
cognitive,1
commercial,1
communication,1
computation,1
computer,2
conclusions,1
contain,,1
credit,1
current,1
data,4
data),,1
data.,1
decision,1
decisions,2
describe,,1
descriptive,1
discovery,,1
disprove,1
draw,1
effects,1
enable,1
enterprise,1
evaluate,1
events,,1
examining,1
extensive,1
field,1
for,1
force,1
fraud,1
gaining,1
given,1
goal,1
harness,1
historical,1
hypotheses.,1
improve,2
improvements,1
in,5
include,1
increasingly,1
industries,1
information,1
information,,1
interpretation,,1
involves,1
is,3
knowledge,1
make,2
management,,1
marketing,2
mathematics.,1
may,1
meaningful,1
methods,1
mix,1
modeling,,2
models,,1
more-informed,1
most,1
of,8
often,

Otra opción para extraer los resultados es usar

      $ hive -S -e 'SELECT * FROM word_counts;' > result.csv
     
     
en donde el archivo `result.txt` se almacena localmente.

In [25]:
!rm -rf output wordcount *.hql *.log